In [188]:
import pandas as pd
import requests
import io
import unidecode
import os
import numpy as np

In [83]:
filenames = ["CasosColombia_01May.csv",
"CasosColombia_03May.csv",
"CasosColombia_04May.csv",
"CasosColombia_05May.csv",
"CasosColombia_10Azbril.csv",
"CasosColombia_11Abril.csv",
"CasosColombia_12Abril.csv",
# "CasosColombia_13Abril.csv",
"CasosColombia_14Abril.csv",
"CasosColombia_15Abril.csv",
"CasosColombia_16Abril.csv",
"CasosColombia_17Abril.csv",
# "CasosColombia_17Abril12pm.csv",
"CasosColombia_18Abril.csv",
"CasosColombia_19Abril.csv",
"CasosColombia_1Abril.csv",
"CasosColombia_20Abril.csv",
"CasosColombia_21Abril.csv",
"CasosColombia_22Abril.csv",
"CasosColombia_23Abril.csv",
"CasosColombia_23marzo.csv",
"CasosColombia_24Abril.csv",
"CasosColombia_24marzo.csv",
"CasosColombia_25Abril.csv",
"CasosColombia_25Marzo.csv",
"CasosColombia_26Abril.csv",
"CasosColombia_26marzo.csv",
"CasosColombia_27Abril.csv",
"CasosColombia_27marzo.csv",
"CasosColombia_28Abril.csv",
"CasosColombia_28marzo.csv",
"CasosColombia_29Abril.csv",
"CasosColombia_29marzo.csv",
"CasosColombia_2Abril.csv",
"CasosColombia_30Abril.csv",
"CasosColombia_30marzo.csv",
"CasosColombia_31marzo.csv",
"CasosColombia_3Abril.csv",
"CasosColombia_4Abril.csv",
"CasosColombia_5Abril.csv",
"CasosColombia_6Abril.csv",
"CasosColombia_7Abril.csv",
"CasosColombia_8Abril.csv",
"CasosColombia_9Abril.csv"]

In [142]:
def contar(df_bog, df_med, df_vill, df_cali,df_col, llave, valor):
    if valor == "infectado":
        bog = df_bog.shape[0]
        med = df_med.shape[0]
        vill = df_vill.shape[0]
        cali = df_cali.shape[0]
        col = df_col.shape[0]
    else:     
        bog = (df_bog[llave] == valor).sum()
        med = (df_med[llave] == valor).sum()
        vill = (df_vill[llave] == valor).sum()
        cali = (df_cali[llave] == valor).sum()
        col = (df_col[llave] == valor).sum()
    return bog, med, vill, cali, col


# Por edades

In [208]:
edad_inferior = list(range(0, 100, 10))
edad_superior = [e + 10 for e in edad_inferior]
edad_inferior.append(0)
edad_superior.append(100)

In [211]:
for i, inferior in enumerate(edad_inferior):
    superior = edad_superior[i]
    bog_all = {}
    med_all = {}
    vill_all = {}
    cali_all = {}
    col_all = {}
    for filename in filenames:
        try:
            url = f"https://raw.githubusercontent.com/guberney/Covid_Colombia/master/Covid_datos/ReporteDiarios/{filename}"
            s=requests.get(url).content
            df=pd.read_csv(io.StringIO(s.decode('utf-8')))
            df = df.rename(columns={"atención":"atencion",
                                    "Atención**": "atencion",
                                    "Atención": "atencion",
                                    "Edad": "edad",
                                    "Ciudad de ubicación": "ciudad",
                                    "Departamento o Distrito ": "Departamento o Distrito"})
            df["edad"] = df["edad"].astype(str)
            df["edad"] = df["edad"].apply(lambda x: np.where(x.isdigit(),x,np.nan))
            
            
            df["edad"] = df["edad"].astype(float)
            
            df = df[((df["edad"] >= inferior) & (df["edad"] < superior))]
            df["atencion"] = df["atencion"].str.upper()
            df["Estado"] = df["Estado"].str.upper()
            df["ciudad"] = df["ciudad"].apply(lambda x: unidecode.unidecode(x.upper()))
            df = df.replace("BOGOTA D.C.", "BOGOTA")

            nombre_fecha = filename
            nombre_fecha = nombre_fecha.replace("CasosColombia_", "").replace(".csv", "")
            nombre_fecha = nombre_fecha.replace("May", "-05").replace("Abril", "-04").replace("marzo", "-03").replace("Marzo", "-03")
            nombre_fecha = nombre_fecha + "-2020"

            df_bog = df[df["ciudad"] == "BOGOTA"]
            df_med = df[df["ciudad"] == "MEDELLIN"]
            df_vill = df[df["ciudad"] == "VILLAVICENCIO"]
            df_cali = df[df["ciudad"] == "CALI"]

            inf_bog, inf_med, inf_vill, inf_cali, inf_col = contar(df_bog, df_med, df_vill, df_cali, df ,"atencion", "infectado")
            rec_bog, rec_med, rec_vill, rec_cali, rec_col = contar(df_bog, df_med, df_vill, df_cali, df,"atencion", "RECUPERADO")
            fall_bog, fall_med, fall_vill, fall_cali, fall_col = contar(df_bog, df_med, df_vill, df_cali, df,"atencion", "FALLECIDO")

            leve_bog, leve_med, leve_vill, leve_cali, leve_col = contar(df_bog, df_med, df_vill, df_cali, df ,"Estado", "LEVE")
            mod_bog, mod_med, mod_vill, mod_cali, mod_col = contar(df_bog, df_med, df_vill, df_cali, df,"Estado", "MODERADO")
            grave_bog, grave_med, grave_vill, grave_cali, grave_col = contar(df_bog, df_med, df_vill, df_cali, df,"Estado", "GRAVE")

            bog_all[nombre_fecha] = [inf_bog, rec_bog, fall_bog, leve_bog, mod_bog, grave_bog]
            med_all[nombre_fecha] = [inf_med, rec_med, fall_med, leve_med, mod_med, grave_med]
            vill_all[nombre_fecha] = [inf_vill, rec_vill, fall_vill, leve_vill, mod_vill, grave_vill]
            cali_all[nombre_fecha] = [inf_cali, rec_cali, fall_cali, leve_cali, mod_cali, grave_cali]
            col_all[nombre_fecha] = [inf_col, rec_col, fall_col, leve_col, mod_col, grave_col]

        except Exception as e:
            print(e)
            print(filename)

    df_bog_all = pd.DataFrame(bog_all).T.rename(columns={0:"infectados", 1:"recuperados", 2:"fallecidos", 3:"leve", 4:"moderado", 5:"grave"})
    df_med_all = pd.DataFrame(med_all).T.rename(columns={0:"infectados", 1:"recuperados", 2:"fallecidos", 3:"leve", 4:"moderado", 5:"grave"})
    df_vill_all = pd.DataFrame(vill_all).T.rename(columns={0:"infectados", 1:"recuperados", 2:"fallecidos", 3:"leve", 4:"moderado", 5:"grave"})
    df_cali_all = pd.DataFrame(cali_all).T.rename(columns={0:"infectados", 1:"recuperados", 2:"fallecidos", 3:"leve", 4:"moderado", 5:"grave"})
    df_col_all = pd.DataFrame(col_all).T.rename(columns={0:"infectados", 1:"recuperados", 2:"fallecidos", 3:"leve", 4:"moderado", 5:"grave"})

    df_bog_all = df_bog_all.reset_index().rename(columns={"index":"fecha"})
    df_med_all = df_med_all.reset_index().rename(columns={"index":"fecha"})
    df_vill_all = df_vill_all.reset_index().rename(columns={"index":"fecha"})
    df_cali_all = df_cali_all.reset_index().rename(columns={"index":"fecha"})
    df_col_all = df_col_all.reset_index().rename(columns={"index":"fecha"})


    df_bog_all["fecha"] = pd.to_datetime(df_bog_all["fecha"],dayfirst=True)
    df_med_all["fecha"] = pd.to_datetime(df_med_all["fecha"],dayfirst=True)
    df_vill_all["fecha"] = pd.to_datetime(df_vill_all["fecha"],dayfirst=True)
    df_cali_all["fecha"] = pd.to_datetime(df_cali_all["fecha"],dayfirst=True)
    df_col_all["fecha"] = pd.to_datetime(df_col_all["fecha"],dayfirst=True)

    df_bog_all = df_bog_all.sort_values(by = "fecha")
    df_med_all = df_med_all.sort_values(by = "fecha")
    df_vill_all = df_vill_all.sort_values(by = "fecha")
    df_cali_all = df_cali_all.sort_values(by = "fecha")
    df_col_all = df_col_all.sort_values(by = "fecha")
    
    nombre_dir = f"por_edades/edad_{inferior}_{superior}"
    os.mkdir(nombre_dir) 
    df_bog_all.to_csv(nombre_dir + f"/df_bog_all_{inferior}_{superior}.csv",index=False)
    df_med_all.to_csv(nombre_dir + f"/df_med_all_{inferior}_{superior}.csv",index=False)
    df_vill_all.to_csv(nombre_dir + f"/df_vill_all_{inferior}_{superior}.csv",index=False)
    df_cali_all.to_csv(nombre_dir + f"/df_cali_all_{inferior}_{superior}.csv",index=False)
    df_col_all.to_csv(nombre_dir + f"/df_col_all_{inferior}_{superior}.csv",index=False)


'edad'
CasosColombia_10Azbril.csv
'Estado'
CasosColombia_11Abril.csv
'Estado'
CasosColombia_12Abril.csv
'Estado'
CasosColombia_14Abril.csv
'Estado'
CasosColombia_15Abril.csv
'Estado'
CasosColombia_16Abril.csv
'Estado'
CasosColombia_17Abril.csv
'Estado'
CasosColombia_18Abril.csv
'Estado'
CasosColombia_1Abril.csv
'Estado'
CasosColombia_23marzo.csv
'Estado'
CasosColombia_24marzo.csv
'Estado'
CasosColombia_25Marzo.csv
'Estado'
CasosColombia_26marzo.csv
'Estado'
CasosColombia_27marzo.csv
'Estado'
CasosColombia_28marzo.csv
'Estado'
CasosColombia_29marzo.csv
'Estado'
CasosColombia_2Abril.csv
'Estado'
CasosColombia_30marzo.csv
'Estado'
CasosColombia_31marzo.csv
'Estado'
CasosColombia_3Abril.csv
'Estado'
CasosColombia_4Abril.csv
'Estado'
CasosColombia_5Abril.csv
'Estado'
CasosColombia_6Abril.csv
'Estado'
CasosColombia_7Abril.csv
'Estado'
CasosColombia_8Abril.csv
'Estado'
CasosColombia_9Abril.csv
'edad'
CasosColombia_10Azbril.csv
'Estado'
CasosColombia_11Abril.csv
'Estado'
CasosColombia_12Abril.

'Estado'
CasosColombia_14Abril.csv
'Estado'
CasosColombia_15Abril.csv
'Estado'
CasosColombia_16Abril.csv
'Estado'
CasosColombia_17Abril.csv
'Estado'
CasosColombia_18Abril.csv
'Estado'
CasosColombia_1Abril.csv
'Estado'
CasosColombia_23marzo.csv
'Estado'
CasosColombia_24marzo.csv
'Estado'
CasosColombia_25Marzo.csv
'Estado'
CasosColombia_26marzo.csv
'Estado'
CasosColombia_27marzo.csv
'Estado'
CasosColombia_28marzo.csv
'Estado'
CasosColombia_29marzo.csv
'Estado'
CasosColombia_2Abril.csv
'Estado'
CasosColombia_30marzo.csv
'Estado'
CasosColombia_31marzo.csv
'Estado'
CasosColombia_3Abril.csv
'Estado'
CasosColombia_4Abril.csv
'Estado'
CasosColombia_5Abril.csv
'Estado'
CasosColombia_6Abril.csv
'Estado'
CasosColombia_7Abril.csv
'Estado'
CasosColombia_8Abril.csv
'Estado'
CasosColombia_9Abril.csv
'edad'
CasosColombia_10Azbril.csv
'Estado'
CasosColombia_11Abril.csv
'Estado'
CasosColombia_12Abril.csv
'Estado'
CasosColombia_14Abril.csv
'Estado'
CasosColombia_15Abril.csv
'Estado'
CasosColombia_16Abril